1. ocr 작업(필요시)을 통한 텍스트 인식<br>
2. 텍스트 및 표 추출
3. <br>
    1. 텍스트를 문장 또는 청크 단위로 분할<br>
    2. 맥락 유지를 위한 청크 중복<br>
    3. 불필요한 정보(해더, 푸터) 삭제<br>
4. <br>
    1. 표 처리. 별도 형식으로 저장(csv, json)<br>
    2. 표 처리. 표와 문장을 결합하여 청크로 저장<br>
5. 최종 청크 생성 : json 형태로 저장

In [3]:
import os
import json
import re

import numpy as np
import pandas as pd
from transformers import GPT2Tokenizer
from PyPDF2 import PdfReader
import pdfplumber

In [5]:
pdf_folder_path = os.path.join(os.getcwd(), 'pdf')
pdf_files = os.listdir(pdf_folder_path)
pdf_files

['61105-60-01-PP-501002_UPDATED PMS FOR PLANT AREA_241220_2_우림.pdf',
 'SGC-3100-LEZ-002 PIPING MATERIAL SPECIFICATION_Rev.B_수정중_이테크.pdf']

In [13]:
def get_pdf_text(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

In [29]:
def get_df_from_nested_list(nested_list, header_row=0, data_row=1):
    headers = nested_list[header_row]
    rows = nested_list[data_row:]
    df = pd.DataFrame(rows, columns=headers)
    return df

In [16]:
def get_pdf_table(pdf_path) :
    with pdfplumber.open(pdf_path) as pdf:
        tables = []
        for page in pdf.pages:
            tables += page.extract_tables()
    return tables

In [22]:
# 이태크로 테스트
pdf_path = pdf_files[1]

In [23]:
# 1. 텍스트 추출
pdf_text = get_pdf_text(os.path.join(pdf_folder_path, pdf_path))

In [24]:
# 2. 테이블 추출
pdf_tables = get_pdf_table(os.path.join(pdf_folder_path, pdf_path))

In [32]:
# 2.1 테이블을 데이터프레임으로 변환
pdf_dfs = []
for i in range(len(pdf_tables)):
    try:
        df = get_df_from_nested_list(pdf_tables[i])
        pdf_dfs.append(df)
    except:
        print('테이블 추출 실패', i)

In [41]:
print(len(pdf_dfs))

97


In [40]:
# 3. 텍스트 전처리